In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Converting all the data into Python list
datadir="/content/drive/My Drive/Dataset"
N=3064
traindata=[]

for i in range(1,N+1):
    filename=str(i)+".mat"
    data=h5py.File(os.path.join(datadir,filename),"r")
    traindata.append(data)

    if i%100==0:
      print(filename)

In [ ]:
print(traindata[0])

<HDF5 file "1.mat" (mode r)>


In [ ]:
import random

random.shuffle(traindata)

NameError: ignored

In [ ]:
#Extracting image data from Python List
trainx=[]
testx=[]
trainy=[]
testy=[]

temp=round(4*N/5)  #using 4/5th of the data as training data

#training data loop (trainx,trainy)
for i in range(temp):
    image=traindata[i]["cjdata"]["image"].value

    if image.shape==(512,512):
        image=np.expand_dims(image,axis=0)
        trainx.append(image)
        
        label=int(traindata[i]["cjdata"]["label"].value)-1
        trainy.append(label)

#testx and testy
for i in range(temp,N):
    image=np.asarray(traindata[i]["cjdata"]["image"].value)
    
    if image.shape==(512,512):
        image=np.expand_dims(image,axis=0)
        testx.append(image)
        
        label=int(traindata[i]["cjdata"]["label"].value)-1
        testy.append(label)



NameError: ignored

In [ ]:
#Converting X list to numpy array #sentdexyoutube dogcats
trainx=np.array(trainx).reshape(-1,512,512,1)
testx=np.array(testx).reshape(-1,512,512,1)

In [ ]:
#Converting Y list to numpy array
trainy=np.array(trainy)
testy=np.array(testy)

In [ ]:
#Saving data as a pickle file (unnecessary step)
import pickle
braindata=[trainx,trainy,testx,testy]

with open("newbraindata.pickle","wb") as file:
  pickle.dump(braindata,file)

In [ ]:
#Loading data from pickle file (unnecesary step if data has been fectched from drive)
import pickle
pickledir="/content/drive/My Drive/newbraindata.pickle"
trainx,trainy,testx,testy=pickle.load(open(pickledir,"rb"))

In [ ]:
np.random.seed(0)

In [ ]:
#Model building starts
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()

In [ ]:
#Initial  BLock of the model
ini_input=keras.Input(shape=(512,512,1),name="image")

x1=layers.Conv2D(64,(22,22),strides=2)(ini_input)
x1=layers.MaxPooling2D((4,4))(x1)
x1=layers.BatchNormalization()(x1)

x2=layers.Conv2D(128,(11,11),strides=2,padding="same")(x1)
x2=layers.MaxPooling2D((2,2))(x2)
x2=layers.BatchNormalization()(x2)

x3=layers.Conv2D(256,(7,7),strides=2,padding="same")(x2)
x3=layers.MaxPooling2D((2,2))(x3)
x3=layers.BatchNormalization()(x3)

x4=layers.Flatten()(x3)
x4=layers.Activation("relu")(x4)

x5=layers.Dense(1024,"relu")(x4)

x6=layers.Dense(256,"relu")(x5)

x9=layers.Dense(3)(x6)
pred=layers.Activation("softmax")(x9)

model=keras.Model(inputs=ini_input,outputs=pred)

model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(trainx,trainy,epochs=20,batch_size=32,shuffle=False)

Epoch 1/20
77/77 [==============================] - 25s 321ms/step - loss: 0.9946 - accuracy: 0.6426
Epoch 2/20
77/77 [==============================] - 25s 321ms/step - loss: 0.5693 - accuracy: 0.7471
Epoch 3/20
77/77 [==============================] - 25s 324ms/step - loss: 0.4501 - accuracy: 0.7971
Epoch 4/20
77/77 [==============================] - 25s 324ms/step - loss: 0.4175 - accuracy: 0.8139
Epoch 5/20
77/77 [==============================] - 25s 325ms/step - loss: 0.3460 - accuracy: 0.8500
Epoch 6/20
77/77 [==============================] - 25s 324ms/step - loss: 0.2860 - accuracy: 0.8873
Epoch 7/20
77/77 [==============================] - 25s 324ms/step - loss: 0.2268 - accuracy: 0.9057
Epoch 8/20
77/77 [==============================] - 25s 324ms/step - loss: 0.1864 - accuracy: 0.9250
Epoch 9/20
77/77 [==============================] - 25s 325ms/step - loss: 0.1307 - accuracy: 0.9512
Epoch 10/20
77/77 [==============================] - 25s 323ms/step - loss: 0.0869 - accura

In [ ]:
model1=keras.models.save_model("CNNforbrainMRI.model")

NameError: ignored

In [ ]:
result=model.evaluate(testx,testy)
print(result)

20/20 [==============================] - 3s 146ms/step - loss: 0.4774 - accuracy: 0.8900
[0.4773988425731659, 0.889983594417572]


In [ ]:
!zip -r /content/CNNforbrainMRI.zip /content/CNNforbrainMRI.model

  adding: content/CNNforbrainMRI.model/ (stored 0%)
  adding: content/CNNforbrainMRI.model/assets/ (stored 0%)
  adding: content/CNNforbrainMRI.model/variables/ (stored 0%)
  adding: content/CNNforbrainMRI.model/variables/variables.data-00000-of-00002 (deflated 82%)
  adding: content/CNNforbrainMRI.model/variables/variables.index (deflated 71%)
  adding: content/CNNforbrainMRI.model/variables/variables.data-00001-of-00002 (deflated 17%)
  adding: content/CNNforbrainMRI.model/saved_model.pb (deflated 90%)


In [ ]:
traincount=[0,0,0]
testcount=[0,0,0]


for i in range(len(trainy)):
  temp=trainy[i]
  traincount[temp]=traincount[temp]+1

for i in range(len(testy)):
  temp=testy[i]
  testcount[temp]=testcount[temp]+1
  
print(traincount)
print(testcount)

[564, 1147, 729]
[144, 279, 186]
